
# Goal:
* Figure out why AP decreases with the number of probes
* Look at the combinations being tested
* Look at the distance values 
* Look at the coarse quantizer
* Look at our reconstruction methods
* Look at baseline methods -- are they sound? Are they making up numbers?

In [2]:
import os
import pickle
import json


import numpy as np
from importlib import reload

In [3]:
import random

import utils
import data
import search_concept
import search_expand

reload(utils)
reload(data)
reload(search_concept)
reload(search_expand)


# Index settings
dim = 512
k_coarse = 512
m = 8
nbits = 8
n_probes_list = [5, 10, 25, 50]
segments_to_test = 50

coco_path = '/data/users/jie/data-slicing/COCO/'
embed_path = '/data/users/jie/data-slicing/COCO/embeds/coco-2014-val-clip-embeds-532'
pickle_path = os.path.join(coco_path, 'embeds/coco-2014-val-clip-embeds-fast-2.pkl')
print(coco_path)
print(pickle_path)
if not os.path.exists(pickle_path):
    print('what')
    print(pickle_path)
embed_dict = utils.create_new_pickle(embed_path, pickle_path)



/data/users/jie/data-slicing/COCO/
/data/users/jie/data-slicing/COCO/embeds/coco-2014-val-clip-embeds-fast-2.pkl


In [4]:
import math

import utils
import data
import search_concept
import search_expand

reload(utils)
reload(data)
reload(search_concept)
reload(search_expand)

def inner_loop(
        index_pq, index_flat_cpu, packd, img_concept_bitmap, 
        all_images, pqq, kmeans, n_probes, embed_dict, average_embeddings,
        n_segments_list):

    precision_results = {
        'our': [],
        'pq': []
    }
    counter_results = {
        'our': [],
        'pq': []
    }
    print('Number of segments:', n_segments_list)
    for n_segments in n_segments_list:
        random_img = random.choice(all_images)
        start_idx, end_idx, _ = embed_dict['img_to_vec_list'][random_img]
        img_embeddings = average_embeddings[start_idx:end_idx]
        random_embedding_idx = random.sample(range(len(img_embeddings)), min(n_segments, len(img_embeddings)))
        
        features = {f'v_{i}': img_embeddings[idx] for i, idx in enumerate(random_embedding_idx)}

        gt_images, _, gt_counter = search_expand.flat_multisearch(
            index_flat_cpu, features, p_k, vec_to_img, 
            max_search_radius=1e6, exclusive_matching=False)
        if len(gt_images) < p_k:
            continue

        our_images, _, our_counter = search_concept.perform_search(
            features, p_k, kmeans, pqq, packd, img_concept_bitmap, all_images, 
            n_probes=n_probes, exclusive_matching=False)
        pq_images, _, pq_counter = search_expand.flat_multisearch(index_pq, features, p_k, vec_to_img, 
            max_search_radius=1e6, exclusive_matching=False)

        our_ap = utils.calculate_avg_precision(gt_images, our_images)
        pq_ap = utils.calculate_avg_precision(gt_images, pq_images)
        precision_results['our'].append(our_ap)
        precision_results['pq'].append(pq_ap)

        counter_results['our'].append(our_counter)
        counter_results['pq'].append(pq_counter)
    
    return precision_results, counter_results

average_embeddings = embed_dict['average_embeddings']
all_precisions = {}
all_counts = {}
for n_probes in n_probes_list:
    # train_sample_size = 10_000
    train_sample_size = int( 128 * math.sqrt(len(average_embeddings)))
    # data_sample_size = 10_000
    train_sample_size = None
    data_sample_size = None

    index_pq, index_flat_cpu, packd, img_concept_bitmap, all_images, pqq, kmeans = data.get_indices(
            dim=dim, k_coarse=k_coarse, m=m, cluster_bits=nbits, 
            n_probes=n_probes, embed_dict=embed_dict, use_custom_pq=False,
            random_seed=None, train_sample_size=train_sample_size,
            data_sample_size=data_sample_size
        )

    print(index_pq.nprobe, 'is number of nprobe i configured')

    all_images = list(embed_dict['img_to_vec_list'].keys())
    vec_to_img = embed_dict['vec_to_img']
    n_iterations = 10
    n_segments_list = [2, 4, 6]
    p_k = 10

    all_counts[n_probes] = {
        'our': [],
        'pq': []
    }

    all_precisions[n_probes] = {
        'our': [],
        'pq': []
    }

    for i in range(n_iterations):
        precisions, counts = inner_loop(
            index_pq, index_flat_cpu, packd, img_concept_bitmap, 
            all_images, pqq, kmeans, n_probes, embed_dict, average_embeddings,
            n_segments_list
        )
        all_counts[n_probes]['our'].append(counts['our'])
        all_counts[n_probes]['pq'].append(counts['pq'])

        all_precisions[n_probes]['our'].append(precisions['our'])
        all_precisions[n_probes]['pq'].append(precisions['pq'])

    print('Number of probes:', n_probes)
    print('Our precision:', np.mean(all_precisions[n_probes]['our']))
    print('PQ precision:', np.mean(all_precisions[n_probes]['pq']))
    print('Our counter:', np.mean(all_counts[n_probes]['our']))
    print('PQ counter:', np.mean(all_counts[n_probes]['pq']))
    print('')

for n_probes in n_probes_list:
    print('Number of probes:', n_probes)
    print('Our precision:', np.mean(all_precisions[n_probes]['our']))
    print('PQ precision:', np.mean(all_precisions[n_probes]['pq']))
    print('Our counter:', np.mean(all_counts[n_probes]['our']))
    print('PQ counter:', np.mean(all_counts[n_probes]['pq']))
    print('')


        


Building IVFPQ index
Building Flat index


WARNING clustering 512 points to 512 centroids: please provide at least 19968 training points


Kmeans trained (hax) 512
521 1024


40501it [00:05, 8099.53it/s] 


5 is number of nprobe i configured
Number of segments: [2, 4, 6]
PQ: Number of candidates 10
PQ: Sorted images ['COCO_val2014_000000000395.jpg', 'COCO_val2014_000000131593.jpg', 'COCO_val2014_000000509451.jpg', 'COCO_val2014_000000205134.jpg', 'COCO_val2014_000000281837.jpg', 'COCO_val2014_000000157516.jpg', 'COCO_val2014_000000130826.jpg', 'COCO_val2014_000000298405.jpg', 'COCO_val2014_000000209018.jpg', 'COCO_val2014_000000170974.jpg']
Query 0 w/: (5,)
Query 1 w/: (5,)
Key v_0 maps to concepts 5
Key v_1 maps to concepts 5
US: Number of matching images 480
US: Distances {9103: 0.45858845114707947, 235: 0.4555501639842987, 27581: 0.4576236456632614, 6646: 0.45188450813293457, 38312: 0.44645020365715027, 14294: 0.45501530170440674, 40414: 0.4432523548603058, 22: 0.27955513447523117, 6736: 0.44593171775341034, 35445: 0.402069553732872}
US: Sorted images ['COCO_val2014_000000000395.jpg', 'COCO_val2014_000000509451.jpg', 'COCO_val2014_000000580621.jpg', 'COCO_val2014_000000096711.jpg', 'CO

...

25 is number of nprobe i configured
Number of segments: [2, 4, 6]
PQ: Number of candidates 27
PQ: Sorted images ['COCO_val2014_000000189766.jpg', 'COCO_val2014_000000274549.jpg', 'COCO_val2014_000000409117.jpg', 'COCO_val2014_000000520508.jpg', 'COCO_val2014_000000221776.jpg', 'COCO_val2014_000000432131.jpg', 'COCO_val2014_000000392022.jpg', 'COCO_val2014_000000134098.jpg', 'COCO_val2014_000000187302.jpg', 'COCO_val2014_000000021905.jpg']
Query 0 w/: (25,)
Query 1 w/: (25,)
Key v_0 maps to concepts 25
Key v_1 maps to concepts 25
US: Number of matching images 7691
US: Distances {30569: 0.27495429664850235, 11389: 0.2732626348733902, 7629: 0.2709636315703392, 26396: 0.2721540778875351, 1557: 0.2666231393814087, 14228: 0.26002301275730133, 28508: 0.26603392884135246, 6767: 0.2671619951725006, 27330: 0.26890289783477783, 13226: 0.19077866524457932}
US: Sorted images ['COCO_val2014_000000189766.jpg', 'COCO_val2014_000000204049.jpg', 'COCO_val2014_000000409117.jpg', 'COCO_val2014_00000002190

WARNING clustering 512 points to 512 centroids: please provide at least 19968 training points


Kmeans trained (hax) 512
521 1024


40501it [00:05, 8015.14it/s] 


50 is number of nprobe i configured
Number of segments: [2, 4, 6]
PQ: Number of candidates 16
PQ: Sorted images ['COCO_val2014_000000185593.jpg', 'COCO_val2014_000000128506.jpg', 'COCO_val2014_000000418578.jpg', 'COCO_val2014_000000385473.jpg', 'COCO_val2014_000000107216.jpg', 'COCO_val2014_000000058522.jpg', 'COCO_val2014_000000077118.jpg', 'COCO_val2014_000000208825.jpg', 'COCO_val2014_000000574509.jpg', 'COCO_val2014_000000183657.jpg']
Query 0 w/: (50,)
Query 1 w/: (50,)
Key v_0 maps to concepts 50
Key v_1 maps to concepts 50
US: Number of matching images 18909
US: Distances {1662: 0.3775612562894821, 22605: 0.3735465258359909, 31333: 0.3772277906537056, 24211: 0.36920125782489777, 34308: 0.3724829852581024, 27043: 0.3586641252040863, 12906: 0.27813248336315155, 12673: 0.358134463429451, 12261: 0.36823485791683197, 31048: 0.3722046837210655}
US: Sorted images ['COCO_val2014_000000185593.jpg', 'COCO_val2014_000000182348.jpg', 'COCO_val2014_000000388014.jpg', 'COCO_val2014_00000017616

In [5]:
import math
import random

import utils
import data
import search_concept
import search_expand
import numpy as np

reload(utils)
reload(data)
reload(search_concept)
reload(search_expand)

def inner_loop(
        index_pq, index_flat_cpu, packd, img_concept_bitmap, 
        all_images, pqq, kmeans, n_probes, embed_dict, average_embeddings,
        n_segments_list):

    precision_results = {
        'our': {n: [] for n in n_segments_list},
        'pq': {n: [] for n in n_segments_list}
    }
    counter_results = {
        'our': {n: [] for n in n_segments_list},
        'pq': {n: [] for n in n_segments_list}
    }
    for n_segments in n_segments_list:
        random_img = random.choice(all_images)
        start_idx, end_idx, _ = embed_dict['img_to_vec_list'][random_img]
        img_embeddings = average_embeddings[start_idx:end_idx]
        random_embedding_idx = random.sample(range(len(img_embeddings)), min(n_segments, len(img_embeddings)))
        
        features = {f'v_{i}': img_embeddings[idx] for i, idx in enumerate(random_embedding_idx)}

        gt_images, _, gt_counter = search_expand.flat_multisearch(
            index_flat_cpu, features, p_k, vec_to_img, 
            max_search_radius=1e6, exclusive_matching=False)
        
        if len(gt_images) < p_k:
            continue

        print('@@@@@ US @@@@@')
        our_images, _, our_counter = search_concept.perform_search(
            features, p_k, kmeans, pqq, packd, img_concept_bitmap, all_images, 
            n_probes=n_probes, exclusive_matching=False)
        print('@@@@@ PQ! @@@@@')
        pq_images, _, pq_counter = search_expand.flat_multisearch(
            index_pq, features, p_k, vec_to_img, 
            max_search_radius=1e6, exclusive_matching=False)
        print('-----------')

        print('GT images', gt_images)
        print('Our images', our_images)
        print('PQ images', pq_images)

        our_ap = utils.calculate_avg_precision(gt_images, our_images)
        pq_ap = utils.calculate_avg_precision(gt_images, pq_images)
        precision_results['our'][n_segments].append(our_ap)
        precision_results['pq'][n_segments].append(pq_ap)

        counter_results['our'][n_segments].append(our_counter)
        counter_results['pq'][n_segments].append(pq_counter)
    
    return precision_results, counter_results

n_probes_list = [25, 50]
average_embeddings = embed_dict['average_embeddings']
all_precisions = {}
all_counts = {}
n_segments_list = [4, 6]

print('Current Settings:')
print('Number of probes:', n_probes_list)
print('Number of segments:', n_segments_list)
print('K-coarse:', k_coarse)
print('M:', m)
print('N-bits:', nbits)
print('-----------')

def bitmap_to_sparse(bitmap):
    from scipy import sparse
    import pandas as pd
    sparse_mat = sparse.csr_matrix(bitmap)
    return pd.DataFrame.sparse.from_spmatrix(sparse_mat).values

for n_probes in n_probes_list:
    train_sample_size = int(128 * math.sqrt(len(embed_dict['img_to_vec_list'])))
    print(train_sample_size, 'is train sample size')
    # data_sample_size = 10_000
    data_sample_size = None
    # data_sample_size = None

    index_pq, index_flat_cpu, packd, img_concept_bitmap, all_images, pqq, kmeans = data.get_indices(
            dim=dim, k_coarse=k_coarse, m=m, cluster_bits=nbits, 
            n_probes=n_probes, embed_dict=embed_dict, use_custom_pq=False,
            random_seed=None, train_sample_size=train_sample_size,
            data_sample_size=data_sample_size
        )

    img_concept_bitmap = bitmap_to_sparse(img_concept_bitmap)

    print(index_pq.nprobe, 'is number of nprobe i configured')
    print('-----------')

    all_images = list(embed_dict['img_to_vec_list'].keys())
    vec_to_img = embed_dict['vec_to_img']
    n_iterations = 1
    p_k = 10

    all_counts[n_probes] = {
        'our': {n: [] for n in n_segments_list},
        'pq': {n: [] for n in n_segments_list}
    }

    all_precisions[n_probes] = {
        'our': {n: [] for n in n_segments_list},
        'pq': {n: [] for n in n_segments_list}
    }

    for i in range(n_iterations):
        precisions, counts = inner_loop(
            index_pq, index_flat_cpu, packd, img_concept_bitmap, 
            all_images, pqq, kmeans, n_probes, embed_dict, average_embeddings,
            n_segments_list
        )
        for n_segments in n_segments_list:
            all_counts[n_probes]['our'][n_segments].extend(counts['our'][n_segments])
            all_counts[n_probes]['pq'][n_segments].extend(counts['pq'][n_segments])
            all_precisions[n_probes]['our'][n_segments].extend(precisions['our'][n_segments])
            all_precisions[n_probes]['pq'][n_segments].extend(precisions['pq'][n_segments])

    print('Number of probes:', n_probes)
    for n_segments in n_segments_list:
        print(f'  Number of segments: {n_segments}')
        print(f'    Our precision: {np.mean(all_precisions[n_probes]["our"][n_segments])}')
        print(f'    PQ precision: {np.mean(all_precisions[n_probes]["pq"][n_segments])}')
        print(f'    Our counter: {np.mean(all_counts[n_probes]["our"][n_segments])}')
        print(f'    PQ counter: {np.mean(all_counts[n_probes]["pq"][n_segments])}')
    print('')

print("Overall results:")
for n_probes in n_probes_list:
    print('Number of probes:', n_probes)
    for n_segments in n_segments_list:
        print(f'  Number of segments: {n_segments}')
        print(f'    Our precision: {np.mean(all_precisions[n_probes]["our"][n_segments])}')
        print(f'    PQ precision: {np.mean(all_precisions[n_probes]["pq"][n_segments])}')
        print(f'    Our counter: {np.mean(all_counts[n_probes]["our"][n_segments])}')
        print(f'    PQ counter: {np.mean(all_counts[n_probes]["pq"][n_segments])}')
    print('')

Current Settings:
Number of probes: [25, 50]
Number of segments: [4, 6]
K-coarse: 512
M: 8
N-bits: 8
-----------
25759 is train sample size
Training on subset 0.052326443552892184
Building IVFPQ index
Building Flat index


WARNING clustering 512 points to 512 centroids: please provide at least 19968 training points


Kmeans trained (hax) 512


40501it [00:00, 55631.59it/s]
/tmp/ipykernel_2756724/1144895585.py:85: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  return pd.DataFrame.sparse.from_spmatrix(sparse_mat).values


25 is number of nprobe i configured
-----------
PQ: Number of candidates 23
PQ: Sorted images ['COCO_val2014_000000472762.jpg', 'COCO_val2014_000000026518.jpg', 'COCO_val2014_000000461681.jpg', 'COCO_val2014_000000056013.jpg', 'COCO_val2014_000000543686.jpg', 'COCO_val2014_000000059108.jpg', 'COCO_val2014_000000578964.jpg', 'COCO_val2014_000000449976.jpg', 'COCO_val2014_000000534178.jpg', 'COCO_val2014_000000362482.jpg']
@@@@@ US @@@@@
Query 0 w/: (25,)
Query 1 w/: (25,)
Query 2 w/: (25,)
Query 3 w/: (25,)
Key v_0 maps to concepts 25
Key v_1 maps to concepts 25
Key v_2 maps to concepts 25
Key v_3 maps to concepts 25
US: Number of matching images 3915
US: Distances {36993: 1.0642732828855515, 38832: 1.063895121216774, 27396: 1.0617684870958328, 7967: 1.059973806142807, 8866: 1.0594891458749771, 32908: 0.6072703301906586, 16198: 1.0061443150043488, 7527: 1.021084040403366, 39073: 1.0441927909851074, 20706: 1.0499426871538162}
US: Sorted images ['COCO_val2014_000000472762.jpg', 'COCO_val2

WARNING clustering 512 points to 512 centroids: please provide at least 19968 training points


Kmeans trained (hax) 512


40501it [00:00, 50115.52it/s] 


50 is number of nprobe i configured
-----------
PQ: Number of candidates 23
PQ: Sorted images ['COCO_val2014_000000221882.jpg', 'COCO_val2014_000000425933.jpg', 'COCO_val2014_000000027917.jpg', 'COCO_val2014_000000048972.jpg', 'COCO_val2014_000000503252.jpg', 'COCO_val2014_000000176193.jpg', 'COCO_val2014_000000373060.jpg', 'COCO_val2014_000000004359.jpg', 'COCO_val2014_000000493815.jpg', 'COCO_val2014_000000245481.jpg']
@@@@@ US @@@@@
Query 0 w/: (50,)
Query 1 w/: (50,)
Query 2 w/: (50,)
Query 3 w/: (50,)
Key v_0 maps to concepts 50
Key v_1 maps to concepts 50
Key v_2 maps to concepts 50
Key v_3 maps to concepts 50
US: Number of matching images 13031
US: Distances {20648: 0.9455460906028748, 7222: 0.936696320772171, 28008: 0.9382471740245819, 3876: 0.9149557203054428, 34336: 0.9110195487737656, 34352: 0.9252044409513474, 26864: 0.894716203212738, 32161: 0.8721569776535034, 15484: 0.5880171060562134, 1994: 0.9082971662282944}
US: Sorted images ['COCO_val2014_000000221882.jpg', 'COCO_va

In [6]:
print("Overall results:")
for n_probes in n_probes_list:
    print('Number of probes:', n_probes)
    for n_segments in n_segments_list:
        print(f'  Number of segments: {n_segments}')
        print(f'    Our precision: {np.mean(all_precisions[n_probes]["our"][n_segments])}')
        print(f'    PQ precision: {np.mean(all_precisions[n_probes]["pq"][n_segments])}')
        print(f'    Our counter: {np.mean(all_counts[n_probes]["our"][n_segments])}')
        print(f'    PQ counter: {np.mean(all_counts[n_probes]["pq"][n_segments])}')
    print('')

Overall results:
Number of probes: 25
  Number of segments: 4
    Our precision: 0.1
    PQ precision: 0.1
    Our counter: 41016.0
    PQ counter: 144623.0
  Number of segments: 6
    Our precision: 0.3930952380952381
    PQ precision: 0.2
    Our counter: 56301.0
    PQ counter: 197740.0

Number of probes: 50
  Number of segments: 4
    Our precision: 0.19285714285714287
    PQ precision: 0.16666666666666666
    Our counter: 169357.0
    PQ counter: 274156.0
  Number of segments: 6
    Our precision: 0.1
    PQ precision: 0.1
    Our counter: 275012.0
    PQ counter: 430354.0



In [2]:
import faiss
import numpy as np
from faiss.contrib.datasets import SyntheticDataset

ds = SyntheticDataset(32, 2000, 1000, 5)

index2 = faiss.index_factory(ds.d, "OPQ4,IVF100,PQ4x8np") 

In [4]:
# manually train the pretransform 
vt = faiss.downcast_VectorTransform(index2.chain.at(0))
vt.train(ds.get_train())

xt_transformed = vt.apply(ds.get_train())

# train the coarse quantizer
index_ivf = faiss.downcast_index(index2.index)
km = faiss.Kmeans(index_ivf.d, index_ivf.nlist, niter=index_ivf.cp.niter)
km.train(xt_transformed)
index_ivf.quantizer.add(km.centroids)   
# after k-means the centroids are added to the quantizer. This applies also to more complex quantizers like HNSW 

# train the PQ quantizer
index_ivf.train(xt_transformed)

WARNING clustering 2000 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 2000 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 2000 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 2000 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 2000 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 2000 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 2000 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 2000 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 2000 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 2000 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 2000 points to 256 centroids: please prov

In [11]:

km2 = faiss.Kmeans(index_ivf.d, index_ivf.nlist, niter=index_ivf.cp.niter)
km2.index.add(km.centroids)


In [17]:
index_ivf.quantizer.reconstruct_n(0, index_ivf.nlist)

array([[ 0.12838224, -0.02205496, -0.03139551, ...,  0.32923406,
         0.3375791 ,  0.3094011 ],
       [ 1.1718087 ,  0.39742315, -0.08726905, ..., -0.26676318,
        -0.42020518,  0.28959188],
       [ 0.00748609, -0.8629916 ,  0.09527918, ...,  0.37539458,
        -0.41916823, -0.11089257],
       ...,
       [ 0.35465205, -0.97603154, -0.05479439, ..., -1.4664073 ,
        -0.25075674,  0.17634328],
       [-1.0891715 ,  0.02416217,  0.15296915, ...,  0.97292536,
         0.8763622 , -0.24081537],
       [ 0.4350278 , -0.35931984,  0.483309  , ..., -0.5475069 ,
        -0.23947744,  0.32009158]], dtype=float32)